# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [ ]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import SimpleReader
import numpy as np
import h5py

import sys
import json
import requests
from IPython.display import IFrame

## Start the server

In [ ]:
%%bash --bg --proc bg_proc
python /Users/ageller/Visualizations/Firefly/FireflyFlaskApp.py

## Show Firefly in an IFrame

In [ ]:
url = "http://localhost:5000/combined"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [ ]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("foo.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to foo.hdf5")

simple = SimpleReader("foo.hdf5",write_jsons_to_disk=False)

## Send this data to the Flask app

*This sends the data.  (Then look back above to see it.)*

In [ ]:
simple.sendDataViaFlask()

In [ ]:
## alternatively, make the request yourself (literally just copying the code from sendDataViaFlask())
port = 5000
print('sending to Firefly', sys.getsizeof(simple.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=simple.JSON)

*Testing with FIRE data*

In [ ]:
port = 5000
fdir = '/Users/ageller/VISUALIZATIONS/FIREdata/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=300)
print('sending to Firefly', sys.getsizeof(reader.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=reader.JSON)

In [ ]:
fdir = '/Users/ageller/VISUALIZATIONS/FIREdata/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=100)

In [ ]:
#check
foo = json.loads(reader.JSON)
for fname in foo:
    print(fname, len(foo[fname]))
    if ('filenames.json' in fname):
        print(foo[fname])

In [ ]:
#send the data to flask
port = 5000
print('sending to Firefly', sys.getsizeof(reader.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=reader.JSON)

*To kill the Firefly server.*

In [ ]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill